_Autor:_    __Jesús Casado__ <br> _Revisión:_ __31/05/2018__ <br>

__Introducción__<br>
Se leen las series diarias de temperatura originales de la AEMET para cada provincia y se crean cuatro archivos (estaciones, Tmax, Tmin y Tmed) con la serie reorganizada.

__Cosas a corregir__ <br>

In [77]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
#import tqdm
import matplotlib.pyplot as plt
from calendar import monthrange
import datetime
#from mpl_toolkits.basemap import Basemap, cm

In [3]:
path = 'F:/Series/AEMET/temperatura_diaria/'
provincias = ['Asturias', 'Leon', 'Palencia']

In [143]:
#provincia = 'Asturias'
for provincia in provincias:
    print(provincia)
    
    # Importar el archivo original de esta provincia
    aux_file = pd.read_csv(path + 'Original/' + provincia + '.csv', sep=';', encoding='latin-1')
    
    # Estaciones
    # ----------
    # Identificar los distintos códigos de 'INDICATIVO'
    stns = aux_file.INDICATIVO.unique()

    # Crear el 'data frame'
    cols = aux_file.columns[0:9]
    cols = np.delete(cols, [1, 2])
    estaciones = pd.DataFrame(index=stns, columns=cols)
    # Completarlo
    for i, stn in enumerate(stns):
        estaciones.iloc[i, :] = aux_file.loc[aux_file.INDICATIVO == stn].iloc[0, [0, 3, 4, 5, 6, 7,8]]

    # Exportar estaciones
    estaciones.to_csv(path + 'Tratado/' + provincia + '_estaciones.csv', index=False)
    
    # Series
    # ------
    # Fecha de inicio de la serie
    start = datetime.date(aux_file.ANO.min(), 1, 1)
    # Fecha de final de la serie
    year_e = aux_file.ANO.max() # último año de la serie
    month_e = aux_file.loc[aux_file.ANO == year_e, 'MES'].max() # mes más elevado del último año de la serie
    end = datetime.date(aux_file.ANO.max(), month_e, monthrange(year_e, month_e)[1])
    # Serie de fechas
    dates = pd.date_range(start, end).date
    del start, end, year_e, month_e

    # Crear los 'data frame' donde se guardará la serie
    Tmax = pd.DataFrame(index=dates, columns=stns)
    Tmax.index.name = 'Fecha'
    print('Nº de días:', Tmax.shape[0], '\tNº de estaciones:', Tmax.shape[1])
    Tmin = Tmax.copy()
    Tmed = Tmax.copy()
    
    # Completar las series
    for j, stn in enumerate(stns):
        # Extraer las filas correspondientes a la estación
        aux_stn = aux_file.loc[aux_file.INDICATIVO == stn, :]
        for i, l in enumerate(aux_stn.index):
            # Extraer año y mes para calcular los días del mes, la fecha de inicio
            year, month = aux_stn.loc[l, 'ANO'], aux_stn.loc[l, 'MES']
            days = monthrange(year, month)[1]
            d = datetime.date(year, month, 1)
            # Índice al que corresponde la fecha de inicio del mes
            ind = Tmax.index.get_loc(d)
            # Completar la serie
            Tmax.iloc[ind:(ind+days), j] = aux_stn.iloc[i, 9:(9+days)].values
            Tmin.iloc[ind:(ind+days), j] = aux_stn.iloc[i, 40:(40+days)].values
    Tmed = (Tmax + Tmin) / 2
    
    # Exportar series
    Tmax.to_csv(path + 'Tratado/' + provincia + '_Tmax.csv', index=True)
    Tmin.to_csv(path + 'Tratado/' + provincia + '_Tmin.csv', index=True)
    Tmed.to_csv(path + 'Tratado/' + provincia + '_Tmed.csv', index=True)
    print()

Asturias
Nº de días: 45441 	Nº de estaciones: 127

Leon
Nº de días: 39233 	Nº de estaciones: 87

Palencia
Nº de días: 31928 	Nº de estaciones: 47

